In [2]:
import random 
import tensorflow as tf
import numpy as np
import os
import sys
import subprocess
from pathlib import Path


In [3]:
# import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


training parameters 

In [4]:
# training parameters 
learning_rate=0.001
training_steps=500
batch_size=32
display_step=20

#network parameters 
num_input_units=80
timesteps= 28*28
num_classes=10
num_hidden_units =200
num_output_units =10


In [5]:
def LIFLayer(n,m,dt,Spike_in,state):
    neurons=n
    inputs=m
    tau_m = 5.0
    v_theta = 1.0
    v_reset = 0.0
    tau_s = 5.0
    tau_refract = 3.0
    # tensor variables 
    V = tf.Variable(tf.constant(0.0,shape=[n,1],dtype=tf.float32),name='V')
    Spike = tf.Variable(tf.constant(v_reset,shape=[n,1],dtype=tf.float32),name='Spike')
    W_rec = tf.Variable(tf.random_normal(shape=[n,n],mean=0.0,stddev=0.1,dtype=tf.float32),name='W_rec')
    W_in = tf.Variable(tf.random_normal(shape=[n,m],mean=0.0,stddev=0.1,dtype=tf.float32),name='W_in')
    G=tf.Variable(tf.ones(shape=[n,1],dtype=tf.float32),name='G')
    S=tf.Variable(tf.zeros(shape=[n,n],dtype=tf.float32),name='S')
    G_in=tf.Variable(tf.ones(shape=[n,1],dtype=tf.float32),name='G_in')
    S_in=tf.Variable(tf.zeros(shape=[n,m],dtype=tf.float32),name='S_in')
    I_syn=tf.Variable(tf.ones(shape=[n,1],dtype=tf.float32),name='I_syn')
    t_reset=tf.Variable(tf.ones(shape=[n,1],dtype=tf.float32),name='t_reset')
    
    @tf.custom_gradient
    def calculate_crossing_op(x):
        x_norm=tf.divide(tf.subtract(x,tf.constant(1.0,shape=[n,1])),tf.constant(1.0,shape=[n,1]))
        temp=tf.greater_equal(x,tf.constant(1.0,shape=[n,1],dtype=tf.float32))
        def grad(dy):            
            return tf.maximum(tf.constant(0.0,dtype=tf.float32),tf.subtract(tf.constant(1.0,dtype=tf.float32),tf.abs(x_norm)))  
        return tf.cast(temp,tf.float32), grad
    Spike=calculate_crossing_op(x)
    v_update=tf.subtract(x,tf.multiply(tf.cast(Spike,tf.float32),tf.constant(1.0,shape=[n,1],dtype=tf.float32)))
    ## update conductance for recurrent spikes 
    dS_op=tf.divide(S,3.0)
    dS_in_op=tf.divide(S_in,tau_s)
    S_temp=tf.subtract(S,dS_op*dt)
    S_in_temp=tf.subtract(S_in,dS_in_op*dt)
    S_op=tf.clip_by_value(S_temp,tf.constant(0.0,shape=[n,n]),tf.constant(100.0,shape=[n,n]))
    S_in_op=tf.clip_by_value(S_in_temp,tf.constant(0.0,shape=[n,m]),tf.constant(100.0,shape=[n,m]))
    Spike_op_float=tf.cast(Spike,tf.float32)
    Spike_in_op_float=tf.cast(Spike_in,tf.float32)
    Spike_ax=tf.clip_by_value(tf.subtract(tf.transpose(tf.tile(Spike_op_float,[1,n])),
                                            tf.eye(n,dtype=tf.float32)),0.0,100.0)

    Spike_in_temp=tf.transpose(tf.tile(Spike_in_op_float,[1,n]))
    Spike_in_ax=Spike_in_temp
    S_update= tf.add(S,Spike_ax)
    S_in_update=tf.add(S_in,Spike_in_ax)
    G_op=tf.reduce_sum(tf.multiply(W_rec,S_update), 1, keepdims=True)
    G_in_op=tf.reduce_sum(tf.multiply(W_in,S_in_update), 1, keepdims=True)
    I_input=tf.add(tf.multiply(G_op,v_update),tf.multiply(G_in_op,v_update))

    ## update voltages
    # find neurons in refractory 
    t_subtract= tf.subtract(t_reset,tf.constant(1.0,shape=[n,1]))
    
    t_margin=tf.clip_by_value(t_subtract,0.0,100)
    t_reset_update=tf.add(t_margin,tf.multiply(Spike,tf.constant(v_theta,shape=[n,1],dtype=tf.float32)))

    eligilible_update=tf.cast(tf.equal(t_reset_update,tf.constant(0.0,shape=[n,1])),tf.float32)
    # update voltage
    dV_op=tf.add(tf.constant(v_reset,shape=[n,1],dtype=tf.float32),tf.multiply(eligilible_update,
                                                                                tf.divide(tf.subtract(I_input,v_update),tau_m)))

    V_out=tf.add(v_update,tf.multiply(dV_op,dt))
    return V_out, Spike

    

In [ ]:
def OutputLayer(k,j,spike_in,dt,state):
    neurons=k
    inputs=j
    tau_m = 5.0
    # tensor variables 
    V_out = tf.contrib.eager.Variable(tf.constant(0.0,shape=[k,1],dtype=tf.float32),name='V_out')
    W_out = tf.contrib.eager.Variable(tf.random_normal(shape=[k,j],mean=0.0,stddev=0.1,dtype=tf.float32),name='W_out')
    G_out= tf.contrib.eager.Variable(tf.ones(shape=[k,1],dtype=tf.float32),name='G_out')
    
    ## update membrane dynamics 
    dv_out=tf.divide(state,tau_m)
    V_temp=tf.subtract(state,dv_out*dt)
    V_update=tf.clip_by_value(V_temp,tf.constant(0.0,shape=[k,1]),tf.constant(100.0,shape=[k,1]))
    spike_in_float=tf.cast(spike_in,tf.float32)
    spike_temp=tf.transpose(tf.tile(spike_in_float,[1,k]))
    weight_update=tf.multiply(W_out,spike_temp)
    G_update=tf.reduce_sum(weight_update, 1, keepdims=True)
    # not sure about this step
    v_membrane = tf.add(V_update,G_update)
    return v_membrane 

define a wrapper of the network to compute the sequence of inputs to outputs 

In [ ]:
def SpkLayer(X):
    sequencelength
    